In [2]:
library(igraph)


Attaching package: 'igraph'

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union



In [123]:
g = read_graph(file = 'facebook_combined.txt',format = c("edgelist"),directed=FALSE)
g$names = V(g)
core_nodes= c(1,108,349,484,1087)

###Defining Subgraph for Node 415
id = 415
g1 = induced.subgraph(g, c(id, neighbors(g,id)))
Nr= which(degree(g1) == 24)



###Defining all three measures
CommonNeighbors = function(i,j,g)
{
    Si = neighbors(g, i)
    Sj = neighbors(g, j)
    return(length(intersection(Si,Sj)))
}


Jaccard = function(i,j,g)
{
    Si = neighbors(g, i)
    Sj = neighbors(g, j)
    common = intersection(Si,Sj)
    Union = union(Si,Sj)
    if(length(Union) == 0) return(0)
    else return(length(common) * (1 / length(Union)))
}


AdamicAdar = function(i,j,g)
{
    Si = neighbors(g, i)
    Sj = neighbors(g, j)
    common = intersection(Si,Sj)
    sum = 0
    for(k in common){
        sum = sum + (1 / log(length(neighbors(g, k)))) 
    }
    return(sum)
}


accuracy_score = function(P, R)
{
    if(length(R) == 0) return(0)
    else return(length(intersect(P,R)) * (1 / length(R)))
}

computeMeasure = function(graph_,count, type)
{
        graph_$names = V(graph_)
        neigh = neighbors(graph = graph_,v = userNode)
        disp_graph = delete.vertices(graph_, neigh)
        disp_graph$names = sort(which(!(graph_$names %in% neigh)))
        commonNeigh = c()
        jaccardMeasure = c()
        AdamicMeasure = c()
        for(i in c(1:vcount(disp_graph))){
            commonNeigh = c(commonNeigh, CommonNeighbors(userNode,disp_graph$names[i],graph_))
            jaccardMeasure = c(jaccardMeasure, Jaccard(userNode,disp_graph$names[i],graph_))
            AdamicMeasure = c(AdamicMeasure, AdamicAdar(userNode,disp_graph$names[i],graph_))
        }
        
        commonNeigh = order(commonNeigh, decreasing = TRUE)
        jaccardMeasure = order(jaccardMeasure, decreasing = TRUE)
        AdamicMeasure = order(AdamicMeasure, decreasing = TRUE)
        
        
        if(type ==1)
            return (c(disp_graph$names[commonNeigh[1:count]]))
        if(type == 2)
            return (c(disp_graph$names[jaccardMeasure[1:count]]))
        if(type == 3)
            return (c(disp_graph$names[AdamicMeasure[1:count]]))

}

In [127]:
print("|NR| =")
print(length(Nr))

[1] "|NR| ="
[1] 11


In [ ]:
id = 415
g1 = induced.subgraph(g, c(id, neighbors(g,id)))
Nr= which(degree(g1) == 24)
resC = c()
resJ = c()
resA = c()
for(userNode in Nr)
{
    for(counter in 1:10){
    g_new = g1
    e = E(g_new)[ from(userNode) ]
    
    deleted_edges = 0
    edges_delete = c()
    deleted_friend = c()
    for(i in 1:length(e)){
        randNum = sample(100,size = 1)
        if(randNum <= 25){
            edges_delete = c(edges_delete,e[i])
            deleted_friend = c(deleted_friend,ends(g_new,e[i])[which(ends(g_new,e[i]) != userNode)])
        }
    }
    deleted_edges = length(edges_delete)
    
    g_new = delete_edges(g_new, edges_delete)
    resultC = computeMeasure(g_new, deleted_edges,1)
    resultJ = computeMeasure(g_new, deleted_edges,2)
    resultA = computeMeasure(g_new, deleted_edges,3)
    
    resultC = accuracy_score(resultC,deleted_friend)
    resultJ = accuracy_score(resultJ,deleted_friend)
    resultA = accuracy_score(resultA,deleted_friend)
    resC = c(resC,resultC)
    resJ = c(resJ,resultJ)
    resA = c(resA,resultA)
    }
}
print(mean(resC))
print(mean(resJ))
print(mean(resA))
